What was done here: Code was used to clean some tables, transforming numerical columns into floats, removing * from cells, and placing with null values. I would use it over and over not repeating the code so maybe I've done it to some tables but I have only the last one registred here.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv('Table3a3b_merged.csv')

In [5]:
print(df.dtypes)

Period                                     object
Fund name                                  object
ABN                                       float64
RSE Regulatory classification              object
Fund type                                  object
                                           ...   
Cash flow adjusted net assets              object
Total assets at the end of period          object
Total liabilities at the end of period     object
Net assets at the end of period            object
One-year\nrate of return                   object
Length: 84, dtype: object


In [8]:
#ONLY KEEP PUBLIC OFFER FUNDS IN THE DATASET
filtered_df = df[df['RSE Regulatory classification'] == 'Public offer']
filtered_df['RSE Regulatory classification'].unique()

array(['Public offer'], dtype=object)

In [15]:
df = filtered_df
df.columns

Index(['Period', 'Fund name', 'ABN', 'RSE Regulatory classification',
       'Fund type', 'RSE Membership base', 'Fund's RSE licensee',
       'RSE licensee ownership type', 'RSE licensee profit status',
       'RSE licensee board structure', 'RSE Balance date', 'Duration',
       'RSE Wind up date', 'Net assets at beginning of period',
       'Employer contributions ', 'of which: Super guarantee contributions',
       'of which: Salary sacrifice contributions', 'Personal contributions',
       'Government co-contributions', 'Low income super contributions',
       'Other member contributions', 'Member contributions',
       'Defined benefit contributions', 'Contribution tax and surcharge',
       'Other members’ benefits flows in', 'Total Members' benefit flows in',
       'Inward rollovers', 'of which: SMSF rollovers',
       'of which: Successor fund transfers', 'Outward rollover',
       'of which: SMSF rollovers.1', 'of which: Successor fund transfers.1',
       'Net rollovers', '

In [16]:
print(df.dtypes)

Period                                     object
Fund name                                  object
ABN                                       float64
RSE Regulatory classification              object
Fund type                                  object
                                           ...   
Cash flow adjusted net assets              object
Total assets at the end of period          object
Total liabilities at the end of period     object
Net assets at the end of period            object
One-year\nrate of return                   object
Length: 84, dtype: object


In [17]:
tofloat_cols = ['Net earnings ',
               'Cash flow adjusted net assets',
               'Net assets at the end of period',
               'One-year\nrate of return'
               ]


In [21]:
#Coding to get floats 
import pandas as pd
import numpy as np
#dataset contains * where values are masked to maintain privacy of members.
#to be able to perform my analysis, they will be replace by the median values of the columns.

# Replace '*' with NaN in the specified columns
for col in tofloat_cols:
    df[col] = df[col].replace('*', np.nan)

# Convert columns to float
df[tofloat_cols] = df[tofloat_cols].astype(float)

# Replace NaN values with the median for each column
median_values = df.median(numeric_only=True)
df[tofloat_cols] = df[tofloat_cols].fillna(median_values)

In [23]:
df.groupby(by='RSE licensee profit status', axis=0)['One-year\nrate of return'].mean()

RSE licensee profit status
For profit status        0.061321
Not for profit status    0.077165
Name: One-year\nrate of return, dtype: float64

In [26]:
df.groupby(by='RSE Membership base', axis=0)['One-year\nrate of return'].mean()

RSE Membership base
Corporate base     0.092196
General base       0.058826
Government base    0.074446
Industry base      0.077012
Name: One-year\nrate of return, dtype: float64

In [29]:
df.groupby(by='RSE licensee ownership type', axis=0)['One-year\nrate of return'].mean()


RSE licensee ownership type
Employer sponsor (non-public sector) ownership    0.058971
Employer sponsor corporation ownership            0.112600
Financial services corporation ownership          0.062404
Nominating organisation ownership                 0.078392
Other ownership type                              0.070067
Public company ownership                          0.079533
Public sector organisation ownership              0.062464
Name: One-year\nrate of return, dtype: float64

I want to encode the categorical datas. So, I can find out if they are good features to my model. 

In [ ]:
import pandas as pd

# Define the columns for which you want to create dummy variables
columns = ['RSE licensee profit status', 'RSE Membership base', 'RSE licensee ownership type']

# Loop through the columns and create dummy variables with descriptive column names
for col in columns:
    dummies = pd.get_dummies(df[col], prefix=f'{col}_dummy')
    df = pd.concat([df, dummies], axis=1)  # Concatenate the dummy variables with the original DataFrame

# Now you have a DataFrame (df) with dummy variables and descriptive column names


In [34]:
# Create dummy variables for categorical column
columns=['RSE licensee profit status', 
         'RSE Membership base',
         'RSE licensee ownership type']
status_dummies = pd.get_dummies(df['RSE licensee profit status'], prefix='status')
df_dummies = pd.concat([df, status_dummies], axis=1)


In [35]:
membership_dummies = pd.get_dummies(df['RSE Membership base'], prefix='status')
membership_dummies.drop(membership_dummies.columns[0], axis=1, inplace=True)

df_2 = pd.concat([df_dummies, membership_dummies], axis=1)

In [36]:
otype_dummies = pd.get_dummies(df['RSE licensee ownership type'], prefix='status')
otype_dummies.drop(otype_dummies.columns[0], axis=1, inplace=True)

df_3 = pd.concat([df_2, otype_dummies], axis=1)

In [38]:
df_3.columns

Index(['Period', 'Fund name', 'ABN', 'RSE Regulatory classification',
       'Fund type', 'RSE Membership base', 'Fund's RSE licensee',
       'RSE licensee ownership type', 'RSE licensee profit status',
       'RSE licensee board structure', 'RSE Balance date', 'Duration',
       'RSE Wind up date', 'Net assets at beginning of period',
       'Employer contributions ', 'of which: Super guarantee contributions',
       'of which: Salary sacrifice contributions', 'Personal contributions',
       'Government co-contributions', 'Low income super contributions',
       'Other member contributions', 'Member contributions',
       'Defined benefit contributions', 'Contribution tax and surcharge',
       'Other members’ benefits flows in', 'Total Members' benefit flows in',
       'Inward rollovers', 'of which: SMSF rollovers',
       'of which: Successor fund transfers', 'Outward rollover',
       'of which: SMSF rollovers.1', 'of which: Successor fund transfers.1',
       'Net rollovers', '

In [41]:
df_3.describe()

,ABN,Duration,Net earnings,Cash flow adjusted net assets,Net assets at the end of period,One-year\nrate of return,status_For profit status,status_Not for profit status,status_General base,status_Government base,status_Industry base,status_Employer sponsor corporation ownership,status_Financial services corporation ownership,status_Nominating organisation ownership,status_Other ownership type,status_Public company ownership,status_Public sector organisation ownership
count,3.308000e+03,3310.000000,3.310000e+03,3.310000e+03,3.310000e+03,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000,3310.000000
mean,5.646628e+10,11.940181,2.728820e+05,4.979449e+06,5.411408e+06,0.058076,0.280665,0.119033,0.269789,0.018429,0.076133,0.004532,0.291843,0.063142,0.022659,0.004532,0.002719
std,2.509568e+10,0.682235,1.479223e+06,1.409478e+07,1.575203e+07,0.079268,0.449392,0.323877,0.443917,0.134517,0.265251,0.067176,0.454679,0.243255,0.148835,0.067176,0.052081
min,1.107438e+10,1.000000,-1.159270e+07,9.000000e+00,-2.000000e+00,-0.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.381382e+10,12.000000,4.987500e+02,2.006268e+05,1.427865e+05,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.758319e+10,12.000000,1.765450e+04,9.346195e+05,8.562740e+05,0.072000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.734356e+10,12.000000,1.439648e+05,2.974404e+06,3.130473e+06,0.105000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,9.939486e+10,12.000000,3.923653e+07,2.503812e+08,2.589842e+08,0.889000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
df_3['Fund name'].value_counts().sort_values(ascending=True)


The First Australian Portable Superannuation Fund             1
Australian Bookmakers Superannuation Fund                     1
Arnold Bloch Leibler Superannuation Fund                      1
Retireinvest Preferred Superannuation Plan - Division II      1
Retireinvest Preferred Superannuation Plan - Division III     1
                                                             ..
Prime Super                                                  19
OneSuper                                                     19
Oasis Superannuation Master Trust                            19
Australian Ethical Retail Superannuation Fund                19
AMG Super                                                    19
Name: Fund name, Length: 395, dtype: int64

In [46]:
df_3['Fund name'].nunique()

395

In [47]:
df=df_3

In [50]:
correlation_matrix = df.corr()
columns_to_remove = ['ABN', 'Duration']
correlation_matrix_filtered = correlation_matrix.drop(columns=columns_to_remove, index=columns_to_remove)
correlation_matrix_filtered

/var/folders/p0/hyllg8rj5_5d3h8g4g3dfnjm0000gn/T/ipykernel_10146/975610273.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


,Net earnings,Cash flow adjusted net assets,Net assets at the end of period,One-year\nrate of return,status_For profit status,status_Not for profit status,status_General base,status_Government base,status_Industry base,status_Employer sponsor corporation ownership,status_Financial services corporation ownership,status_Nominating organisation ownership,status_Other ownership type,status_Public company ownership,status_Public sector organisation ownership
Net earnings,1.000000,0.456237,0.510945,0.265496,0.021604,0.227904,0.065672,0.186706,0.107418,0.022606,0.024896,0.198320,0.059848,0.085950,0.107293
Cash flow adjusted net assets,0.456237,1.000000,0.989687,-0.011472,0.033347,0.322261,0.120837,0.218002,0.139175,0.006801,0.035527,0.250817,0.137243,0.140295,0.150981
Net assets at the end of period,0.510945,0.989687,1.000000,0.011686,0.029601,0.327457,0.119571,0.231004,0.136273,0.007363,0.031685,0.250580,0.142283,0.135634,0.180522
One-year\nrate of return,0.265496,-0.011472,0.011686,1.000000,0.025574,0.088536,0.005756,0.028301,0.068587,0.046417,0.035055,0.066549,0.023036,0.018267,0.002891
status_For profit status,0.021604,0.033347,0.029601,0.025574,1.000000,-0.229606,0.864033,-0.085589,-0.151424,-0.042145,0.949348,-0.153869,-0.036371,-0.042145,-0.032616
status_Not for profit status,0.227904,0.322261,0.327457,0.088536,-0.229606,1.000000,-0.072090,0.372765,0.742262,0.183554,-0.127208,0.694758,0.332727,0.183554,0.142051
status_General base,0.065672,0.120837,0.119571,0.005756,0.864033,-0.072090,1.000000,-0.083287,-0.174489,-0.010609,0.815084,-0.051454,0.085835,-0.041011,-0.031738
status_Government base,0.186706,0.218002,0.231004,0.028301,-0.085589,0.372765,-0.083287,1.000000,-0.039334,-0.009245,-0.073140,0.250731,0.250839,-0.009245,0.381073
status_Industry base,0.107418,0.139175,0.136273,0.068587,-0.151424,0.742262,-0.174489,-0.039334,1.000000,0.133275,-0.056491,0.581186,0.094079,0.235037,-0.014989
status_Employer sponsor corporation ownership,0.022606,0.006801,0.007363,0.046417,-0.042145,0.183554,-0.010609,-0.009245,0.133275,1.000000,-0.043314,-0.017516,-0.010273,-0.004552,-0.003523
